In [ ]:
pip install requests

In [ ]:
import requests

class Artist():
    def __init__(self, artist, popularity):
        self.artist = artist
        self.popularity = popularity
        self.events = []

    def get_events(self):
        ticketmaster_baseurl = "https://app.ticketmaster.com/discovery/v1/events.json"
        response = requests.get(ticketmaster_baseurl, params={
            'apikey': 'UVnb1hGAEPt7pAYAY3DD0mj5FG7gxocc',
            'keyword': self.artist
        })
        r = response.json()
        if '_embedded' in r:
            for x in r['_embedded']['events']:
                name = x['name']
                location = x['_embedded']['venues'][0]['name']
                start_date = x['dates']['start']['localDate']
                self.events.append((name, location, start_date))

    def pretty_print(self):
        self.get_events()
        print(f"The artist {self.artist} has a popularity level of {self.popularity}.")
        if self.events:
            for event in self.events:
                if event:
                    print_string = f"The event {event[0]} at {event[1]} is on {event[2]}."
                    print(print_string)

def get_related_artists(aid):
    response = requests.get(f'https://api.spotify.com/v1/artists/{aid}/related-artists')
    r = response.json()
    artists = []
    for artist in r['artists']:
        name = artist['name']
        popularity = artist['popularity']
        artists.append(Artist(name, popularity))
    new_list = [artist for artist in artists if artist.popularity > 60]
    return new_list

def get_artist_id(artist_name):
    response = requests.get('https://api.spotify.com/v1/search', params={
        'q': artist_name,
        'type': 'artist'
    })
    r = response.json()
    return r['artists']['items'][0]['id']

def main():
    try:
        print("Accessing Spotify API")
        artist_name = input('Enter an artist: ')
        artist_id = get_artist_id(artist_name)
        related_artists = get_related_artists(artist_id)

        for artist in related_artists:
            artist.get_events()

        sorted_artists = sorted(related_artists, key=lambda x: x.popularity, reverse=True)

        for artist in sorted_artists:
            artist.pretty_print()
    except ValueError:
        print('Program Failed')

main()



Accessing Spotify API
Enter an artist: Taylor Swift


KeyError: 'artists'

In [ ]:
import requests
import pandas as pd

# Configura tu clave de API de Ticketmaster
API_KEY = "wmvDIXRHQ9zv3zWbdJfGs3ORKDZG291i"


# Inicializa variables
page_size = 10 # Número de eventos por página
current_page = 0  # Página actual

# Crea una lista para almacenar todos los datos de eventos
all_events_data = []

while True:
    # Construye la URL para buscar eventos con paginación
    search_url = f"https://app.ticketmaster.com/discovery/v2/events.json?apikey={API_KEY}&size={page_size}&page={current_page}"

    # Realiza la solicitud GET a la API de Ticketmaster para buscar eventos
    search_response = requests.get(search_url)

    # Verifica si la solicitud de búsqueda fue exitosa (código de estado 200)
    if search_response.status_code == 200:
        # Convierte la respuesta JSON a un objeto Python
        search_data = search_response.json()

        # Asegúrate de que la respuesta contiene información sobre los eventos
        if "_embedded" in search_data and "events" in search_data["_embedded"]:
            # Itera sobre los eventos y extrae los campos deseados
            for event in search_data["_embedded"]["events"]:
                event_data = {
                    "Event Name": event.get("name", ""),
                    "Venue Name": event["_embedded"]["venues"][0].get("name", ""),
                    "City": event["_embedded"]["venues"][0]["city"].get("name", ""),
                    "Type": event.get("type", ""),
                    "Distance": event.get("distance", ""),
                    "Units": event.get("units", ""),
                    "Location": event.get("location", {}).get("name", ""),
                    "ID": event.get("id", ""),
                    "Locale": event.get("locale", ""),
                    "Description": event.get("description", ""),
                    "Additional Info": event.get("additionalInfo", ""),
                    "URL": event.get("url", ""),
                    "Dates": event.get("dates", {}),
                    "Sales": event.get("sales", {}),
                    "Info": event.get("info", ""),
                    "Please Note": event.get("pleaseNote", ""),
                    "Price Ranges": event.get("priceRanges", []),
                    "Promoter": event.get("promoter", {}),
                    "Promoters": event.get("promoters", []),
                    "Outlets": event.get("outlets", []),
                    "Product Type": event.get("productType", ""),
                    "Products": event.get("products", []),
                    "Seatmap": event.get("seatmap", {}),
                    "Accessibility": event.get("accessibility", {}),
                    "Ticket Limit": event.get("ticketLimit", {}),
                    "Classifications": event.get("classifications", []),
                    "Place": event.get("place", {}),
                    "External Links": event.get("externalLinks", {}),
                    "Test": event.get("test", False),
                    "Aliases": event.get("aliases", []),
                    "Localized Aliases": event.get("localizedAliases", {})
                }
                all_events_data.append(event_data)

            # Incrementa la página actual para obtener la siguiente página en la siguiente iteración
            current_page += 1
        else:
            # No hay más eventos, sal del bucle
            break
    else:
        # Si la solicitud no fue exitosa, imprime el código de estado y el contenido de error
        print(f"Error {search_response.status_code}: {search_response.text}")
        break

# Convierte la lista de eventos a un DataFrame
df = pd.DataFrame(all_events_data)

# Muestra el DataFrame con los datos de los eventos
df


Error 429: {"fault":{"faultstring":"Rate limit quota violation. Quota limit  exceeded. Identifier : wmvDIXRHQ9zv3zWbdJfGs3ORKDZG291i","detail":{"errorcode":"policies.ratelimit.QuotaViolation"}}}


,Event Name,Venue Name,City,Type,Distance,Units,Location,ID,Locale,Description,...,Products,Seatmap,Accessibility,Ticket Limit,Classifications,Place,External Links,Test,Aliases,Localized Aliases
0,Golden State Warriors vs. Phoenix Suns,Chase Center,San Francisco,event,,,,G5vYZ9svB7-aF,en-us,,...,"[{'name': 'PARKWHIZ CHASE CENTER', 'id': 'G5vY...",{'staticUrl': 'https://maps.ticketmaster.com/m...,{'ticketLimit': 4},{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",{},{},False,[],{}
1,Phoenix Suns vs. Milwaukee Bucks,Footprint Center,Phoenix,event,,,,G5v0Z9Yc3P8Mr,en-us,,...,"[{'name': 'PARKWHIZ FOOTPRINT CENTER', 'id': '...",{'staticUrl': 'https://maps.ticketmaster.com/m...,{'ticketLimit': 4},{'info': 'There is an overall four (4) ticket ...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...",{},{},False,[],{}
2,Phoenix Suns vs. Utah Jazz,Footprint Center,Phoenix,event,,,,G5v0Z9Yc3JZMP,en-us,,...,"[{'name': 'PARKWHIZ FOOTPRINT CENTER', 'id': '...",{'staticUrl': 'https://maps.ticketmaster.com/m...,{'ticketLimit': 4},{'info': 'There is an overall four (4) ticket ...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...",{},{},False,[],{}
3,Phoenix Suns vs. Sacramento Kings,Footprint Center,Phoenix,event,,,,G5v0Z9Yc3JYMI,en-us,,...,"[{'name': 'PARKWHIZ FOOTPRINT CENTER', 'id': '...",{'staticUrl': 'https://maps.ticketmaster.com/m...,{'ticketLimit': 4},{'info': 'There is an overall four (4) ticket ...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...",{},{},False,[],{}
4,Milwaukee Bucks vs. Phoenix Suns,Fiserv Forum,Milwaukee,event,,,,vvG1jZ9sTtWKwe,en-us,,...,[],{'staticUrl': 'https://maps.ticketmaster.com/m...,{'info': 'Accessible seating is available for ...,{'info': 'There is an overall 6 ticket limit f...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...",{},{},False,[],{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,Hamilton (Touring),ASU Gammage,Tempe,event,,,,G5v0Z9MQ_6v2H,en-us,,...,[],{'staticUrl': 'https://maps.ticketmaster.com/m...,{'ticketLimit': 4},{'info': 'There is an eight (8) ticket limit f...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...",{},{},False,[],{}
436,Pittsburgh Penguins vs. Columbus Blue Jackets,PPG Paints Arena,Pittsburgh,event,,,,G5vbZ9cLQZp2h,en-us,,...,"[{'name': 'PARKWHIZ PPG PAINTS ARENA', 'id': '...",{'staticUrl': 'https://maps.ticketmaster.com/m...,{'ticketLimit': 8},{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",{},{},False,[],{}
437,Harry Potter and the Cursed Child,Lyric Theatre - NY,New York,event,,,,Z7r9jZ1A7Z9-b,en-us,,...,[],{'staticUrl': 'https://content.resale.ticketma...,{},{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",{},{},False,[],{}
438,Hamilton (Touring),ASU Gammage,Tempe,event,,,,G5v0Z9MQ9x-FJ,en-us,,...,[],{'staticUrl': 'https://maps.ticketmaster.com/m...,{'ticketLimit': 4},{'info': 'There is an eight (8) ticket limit f...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...",{},{},False,[],{}


In [ ]:
# para guardar en csv
from pathlib import Path
path = Path.cwd()
path_procesado = Path(path,"procesado")
path_procesado.mkdir(exist_ok=True)
fichero = Path(path_procesado,"ticketmaster.csv")

# Guarda el DataFrame como un archivo CSV
df.to_csv(fichero, index=False)

NameError: name 'df' is not defined

In [ ]:
import requests

def get_popular_artists():
    response = requests.get('https://api.spotify.com/v1/browse/new-releases')
    if response.status_code == 200:
        data = response.json()
        artists = []
        for item in data.get('albums', {}).get('items', []):
            for artist in item.get('artists', []):
                artists.append(artist['name'])
        return artists
    else:
        print("Error al obtener artistas:", response.status_code)

# Obtener la lista de artistas populares
popular_artists = get_popular_artists()

# Imprimir la lista de artistas
print("Lista de artistas populares:")
for artist in popular_artists:
    print(artist)


Error al obtener artistas: 401
Lista de artistas populares:


TypeError: 'NoneType' object is not iterable

In [ ]:
import requests

# Configura tu clave de API de Last.fm
LASTFM_API_KEY = "d3668e7b9ace955aaefafa6e262386ba"
ARTIST_NAME = "Taylor Swift"
DATE = "04-02-2024"  # Ajusta según sea necesario

# Construye la URL para solicitar datos de Last.fm
url = f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={ARTIST_NAME}&api_key={LASTFM_API_KEY}&format=json"

# Realiza la solicitud a la API de Last.fm
response = requests.get(url)
data = response.json()

# Extrae el número de oyentes en la fecha específica (si está disponible en la respuesta)
if "artist" in data and "stats" in data["artist"]:
    listeners = data["artist"]["stats"]["listeners"]
    print(f"Número de oyentes de {ARTIST_NAME} en {DATE}: {listeners}")
else:
    print(f"No se encontró información para {ARTIST_NAME} en la fecha {DATE}.")


Número de oyentes de Taylor Swift en 04-02-2024: 4473755


In [ ]:
import requests

# Tu API key y secret obtenidos en el paso anterior
client_id = '3e082f0f2dd240c1beb66c9705a663a5'
client_secret = 'd3dd1f7886eb4ba4aa5a76c9095120d7'

# URL de la API de Spotify para obtener un token de acceso
token_url = 'https://accounts.spotify.com/api/token'

# Parámetros necesarios para obtener el token de acceso
token_params = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
}

# Obtener el token de acceso
token_response = requests.post(token_url, data=token_params)
token_data = token_response.json()

# Verificar si se obtuvo correctamente el token
if 'access_token' in token_data:
    access_token = token_data['access_token']

    # Ejemplo de búsqueda de una canción por nombre
    search_url = 'https://api.spotify.com/v1/search'
    query = 'nombre_de_la_cancion'
    search_params = {
        'q': query,
        'type': 'track',
    }

    # Configurar la cabecera con el token de acceso
    headers = {
        'Authorization': f'Bearer {access_token}',
    }

    # Hacer la solicitud de búsqueda
    search_response = requests.get(search_url, params=search_params, headers=headers)
    search_data = search_response.json()

    # Imprimir los resultados de la búsqueda
    print(token_data)

else:
    print('Error al obtener el token de acceso')
    print(token_data)


{'access_token': 'BQASMLN34tE8SJpV0STDjGYdBoXfT27U9zK5QgaZcZkxbgx8tq3HCJAURY77QJbQ-FASFGJePD7iGmGthMVsK2kRSkfCUZ9TwrzGrl6AsUxHb4v4lbw', 'token_type': 'Bearer', 'expires_in': 3600}


In [ ]:
import requests

# Tu API key y secret obtenidos en el paso anterior
client_id = '3e082f0f2dd240c1beb66c9705a663a5'
client_secret = 'd3dd1f7886eb4ba4aa5a76c9095120d7'

# URL de la API de Spotify para obtener un token de acceso
token_url = 'https://accounts.spotify.com/api/token'

# Parámetros necesarios para obtener el token de acceso
token_params = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
}

# Obtener el token de acceso
token_response = requests.post(token_url, data=token_params)
token_data = token_response.json()

# Verificar si se obtuvo correctamente el token
if 'access_token' in token_data:
    access_token = token_data['access_token']

    # Nombre del artista que deseas buscar
    artist_name = 'Taylor Swift'

    # URL de la API de Spotify para buscar al artista
    search_url = 'https://api.spotify.com/v1/search'
    search_params = {
        'q': artist_name,
        'type': 'artist',
    }

    # Configurar la cabecera con el token de acceso
    headers = {
        'Authorization': f'Bearer {access_token}',
    }

    # Hacer la solicitud de búsqueda del artista
    search_response = requests.get(search_url, params=search_params, headers=headers)
    search_data = search_response.json()

    # Obtener el ID del primer resultado (artista)
    if 'artists' in search_data and 'items' in search_data['artists'] and len(search_data['artists']['items']) > 0:
        artist_id = search_data['artists']['items'][0]['id']
        print(f'El ID del artista {artist_name} es: {artist_id}')

    else:
        print(f'No se encontró al artista {artist_name}')

else:
    print('Error al obtener el token de acceso')
    print(token_data)


El ID del artista Taylor Swift es: 06HL4z0CvFAxyc27GXpf02


In [ ]:
import requests

# Tu API key y secret obtenidos en el paso anterior
client_id = '3e082f0f2dd240c1beb66c9705a663a5'
client_secret = 'd3dd1f7886eb4ba4aa5a76c9095120d7'

# URL de la API de Spotify para obtener un token de acceso
token_url = 'https://accounts.spotify.com/api/token'

# Parámetros necesarios para obtener el token de acceso
token_params = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
}

# Obtener el token de acceso
token_response = requests.post(token_url, data=token_params)
token_data = token_response.json()

# Verificar si se obtuvo correctamente el token
if 'access_token' in token_data:
    access_token = token_data['access_token']

    # ID del artista que deseas consultar (puedes obtenerlo de la API de búsqueda)
    artist_id = '06HL4z0CvFAxyc27GXpf02'

    # URL de la API de Spotify para obtener información del artista
    artist_url = f'https://api.spotify.com/v1/artists/{artist_id}'

    # Configurar la cabecera con el token de acceso
    headers = {
        'Authorization': f'Bearer {access_token}',
    }

    # Hacer la solicitud para obtener información del artista
    artist_response = requests.get(artist_url, headers=headers)
    artist_data = artist_response.json()

    # Verificar si se obtuvo correctamente la información del artista
    if 'followers' in artist_data:
        followers_count = artist_data['followers']['total']
        print(f'El artista tiene {followers_count} oyentes mensuales.')

    else:
        print('Error al obtener información del artista')
        print(artist_data)

else:
    print('Error al obtener el token de acceso')
    print(token_data)


El artista tiene 100795465 oyentes mensuales.


In [ ]:
import requests

# Tu API key y secret obtenidos en el paso anterior
client_id = '3e082f0f2dd240c1beb66c9705a663a5'
client_secret = 'd3dd1f7886eb4ba4aa5a76c9095120d7'

# URL de la API de Spotify para obtener un token de acceso
token_url = 'https://accounts.spotify.com/api/token'

# Parámetros necesarios para obtener el token de acceso
token_params = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
}

# Obtener el token de acceso
token_response = requests.post(token_url, data=token_params)
token_data = token_response.json()

# Verificar si se obtuvo correctamente el token
if 'access_token' in token_data:
    access_token = token_data['access_token']

    # Nombre del artista que deseas buscar
    artist_name = 'The Weeknd'

    # URL de la API de Spotify para buscar al artista
    search_url = 'https://api.spotify.com/v1/search'
    search_params = {
        'q': artist_name,
        'type': 'artist',
    }

    # Configurar la cabecera con el token de acceso
    headers = {
        'Authorization': f'Bearer {access_token}',
    }

    # Hacer la solicitud de búsqueda del artista
    search_response = requests.get(search_url, params=search_params, headers=headers)
    search_data = search_response.json()

    # Filtrar los resultados por año (por ejemplo, 2022)
    if 'artists' in search_data and 'items' in search_data['artists']:
        filtered_artists = [artist for artist in search_data['artists']['items'] if 'release_date' in artist and artist['release_date'].startswith('2022')]
        if filtered_artists:
            for artist in filtered_artists:
                print(f"ID del artista {artist_name}: {artist['id']}")
        else:
            print(f"No se encontraron resultados para {artist_name} en 2022")
    else:
        print(f'No se encontró al artista {artist_name}')

else:
    print('Error al obtener el token de acceso')
    print(token_data)


No se encontraron resultados para The Weeknd en 2022


In [ ]:
import requests
import pandas as pd

url = "https://app.ticketmaster.com/discovery/v2/events.json"
API_KEY = "wmvDIXRHQ9zv3zWbdJfGs3ORKDZG291i"

params = {
    "apikey": API_KEY,
    "classificationName": "Music",  # Filtrar solo eventos de música (conciertos)
    "size": 100
}

# Realizar la solicitud GET con los parámetros
response = requests.get(url, params=params)

# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Convertir la respuesta JSON en un diccionario de Python
    data = response.json()

    # Crear una lista para almacenar los detalles de los eventos que no son festivales
    non_festival_details = []

    # Obtener detalles de los eventos
    for event in data["_embedded"]["events"]:
        #if len(non_festival_details) < 100:  # Verificar si se han recopilado menos de 100 eventos
            # Verificar si el nombre del evento no contiene la palabra "festival"
            if "festival" not in event["name"].lower():
                event_name = event["name"]
                event_date = event["dates"]["start"]["localDate"]
                if "localTime" in event["dates"]["start"]:
                    event_time = event["dates"]["start"]["localTime"]
                else:
                    event_time = "Hora no especificada"
                if "priceRanges" in event:
                    if "min" in event["priceRanges"][0]:
                        price_min = event["priceRanges"][0]["min"]
                    else:
                        price_min = "NA"
                    if "max" in event["priceRanges"][0]:
                        price_max = event["priceRanges"][0]["max"]
                    else:
                        price_max = None
                else:
                    price_min = "NA"
                    price_max = None

                non_festival_details.append([event_name, event_date, event_time, price_min, price_max])
        # else:
        #     break  # Detener el bucle una vez que se hayan recopilado 100 eventos

    # Crear DataFrame
    df = pd.DataFrame(non_festival_details, columns=["Nombre del Evento", "Fecha del Evento", "Hora del Evento", "Precio Mínimo", "Precio Máximo"])

    # Imprimir el DataFrame
else:
    print("Error al realizar la solicitud:", response.status_code)


df


,Nombre del Evento,Fecha del Evento,Hora del Evento,Precio Mínimo,Precio Máximo
0,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-02-17,20:00:00,195.0,195.00
1,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-02-18,20:00:00,195.0,195.00
2,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-03-02,20:00:00,195.0,195.00
3,P!NK: Summer Carnival 2024,2024-08-21,18:30:00,59.11,340.00
4,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-02-15,20:00:00,195.0,195.00
...,...,...,...,...,...
95,P!NK - VIP Ticket Experiences,2024-06-21,17:00:00,NA,NaN
96,P!NK - Summer Carnival 2024,2024-06-21,17:00:00,97.05,176.75
97,P!NK - Summer Carnival 2024,2024-06-24,16:30:00,50.0,160.00
98,P!NK - Summer Carnival 2024,2024-06-25,16:30:00,50.0,160.00
